In [1]:
import numpy as np
import random
from typing import Callable
from copy import deepcopy
import matplotlib.pyplot as plt

# Zadanie 2
## Damian Baraniak 324851
#### WSI-24L-G104
Celem zadania jest implementacja algorymtu ewolucyjnego oraz weryfikacja jego działania na przykładzie minimalizacji dwóch funckcji dwuwymiarowych.
- Funkcja Himmelblau:
    $$ f(x,y)=(x^2+y-11)^2+(x+y^2-7)^2 $$ 
- Funckcja Ackleya w wersji dwuwymiarowej:
    $$ f(x,y) = -20\exp(-0.2\sqrt{0.5(x^2+y^2)})-\exp(0.5(\cos2\pi x+\cos2\pi y))+20+e $$

In [2]:
def himmelblau_function(x:float,y:float)->float:
    elem1 = (x**2 + y - 11)**2 
    elem2 = (x + y**2 - 7)**2
    return elem1+elem2

def ackley_function(x:float,y:float)->float:
    elem1 = np.exp(-0.2*np.sqrt(0.5*(x**2+y**2)))
    elem2 = np.exp(0.5*(np.cos(2*np.pi*x)+np.cos(2*np.pi*y)))
    return -20*elem1-elem2+20+np.e

## Struktura algorytmu
Algorytm do poprawnego działania potrzebuje szeregu parametrów. Są to:
- funckcja oceny - funkcja oceniająca osobniki, który najlepiej spełnia wymagania, w naszym przypadku jest to minimalizowanie podanych funkcji
- populacja początkowa - określona grupa osobników początkowych, wygenerowana losowo w pewnym obszarze
- prawdopodobnieństwo krzyżowania - określa jak często będzie dochodzić do krzyżowania między osobnikami
- siła mutacji - określa jak bardzo cechy osobnika mogą się zmieniać w wyniku mutacji
- liczba iteracji - ile razy należy powtórzyć proces ewolucji

### Osobnik 
Do działania algorytmu potrzebny jest osobnik który może podlegać ocenie, mutacji i krzyżowaniu w tym przypadku jest to dwu-wymiarowy wektor $[x,y]$

### Proces
Cały proces zaczyna się na wylosowaniu początkowej populacji oraz oceny wszystkich osobników, następnie powtarza się:
- reprodukcje czyli wybór osobników do dalszych działań
- krzyżowanie osobników
- mutacje osobników
- ocene powstałych osobników
- porównanie najlepszych osobników
- wybranie osobników przchodzących do następnej iteracji

W zadaniu wybrano metody: reprodukcji turniejowej, krzyżowania uśredniającego, mutacji rozkładem normalnym oraz sukcesje elitarną o elicie wielkości 1.


### Reprodukcja turniejowa
Z aktualnej populacji losuje się ze zwracaniem osobniki do grupy turniejowej, następnie z grupy wybiera takiego osobnika, którego cechy dają najlepszy wynik w tym przypadku, któego wartość jest najmniejsza. Proces jest powtarzany, aż powstanie nowa populacja o takim samym rozmiarze. Losowanie ze zwracaniem może powodować, że niektóre, lepsze osobniki znajdą się w nowej populacji kilka razy, dodatkowo każdy osobnik ma jakąś szasnę przetrwania, nawet najgorszy jeśli wylosuj sam siebie do turnieju.

In [3]:
def tournament_selection(population,competition):
    temp_population = []
    while len(temp_population)<len(population):
        # losowanie grup turniejowych
        group = random.choices(population,k=competition)
        # wybór najlepszego osobnika
        temp_population.append(min(group,key=lambda ind:ind.value))
    return temp_population

### Sukcjesja elitarna
Sukcesja elitarna polega na doklejeniu do aktualnej populacji $\eta$ najlepszych osobników z porzedniej populacji, a następnie odrzuceniu $\eta$ najgorszych osobników, aby końcowy rozmiar populacji nie uległ zmianie. Zbyt duża ilość osobników doklejanych może sprawić utknięcie w ekstremach lokalnych, dlatego postanowiono ustanowić rozmiar elity jako $\eta = 1$.

In [4]:
def succession_phase(population,best):
    population.append(deepcopy(best))
    # sortowanie populacji rosnąco względem wartości funkcji dla osobnika
    population.sort(key = lambda ind:ind.value)
    population.pop()

### Krzyżowanie uśredniające i mutacja
Mutacja osobników jest siłą algorytmu ewolucyjnego, a nie krzyżowanie, mimo wszystko zostało tutaj zaimplementowane. Dla każdego osobnika w populacji jest losowane czy dojdzie do krzyżowania, jeśli nie to taki osobnik przechodzi dalej. Jeśli dojdzie do krzyżowania, losowany jest kolejny rodzic, następnie ich cechy są uśredniane przy pomocy wag: $k_t = w_t\cdot R_{t1}+(1-w_t)\cdot R_{t2}$, dla każdej cechy losowana jest osobna waga.
W algorytmie ewolucyjnym każdy osobnik przechodzi mutacje, jest to zmiana wszystkich cech osobnika: $x_{t} = x_t + \sigma \cdot N(0,1)$, $\sigma$ oznacza tutaj siłę mutacji, czyli jak bardzo osobnik może się zmienić.

#### Implementacja osobnika z możliwością porównania osobników, operatorami mutacji oraz krzyżowania

In [5]:
class Individual:
    def __init__(self,position):
        self.position = position
        self.size = len(position)
        self.value = None

    def evaluate(self, func):
        self.value = func(*self.position)
    
    def __lt__(self,other):
        return self.value<other.value
      
    def __str__(self) -> str:
        return f"Position: {self.position}, value: {self.value}"
    
    def mutate(self, strength):
        temp_position = []
        for gene in self.position:
            temp_position.append(gene + strength*random.gauss(0,1))
        self.position = temp_position
    
    def crossover(self,other):
        assert self.size == other.size
        result_position = []
        for i in range(self.size):    
            weight = random.random()
            gene = self.position[i]*weight+other.position[i]*(1-weight)
            result_position.append(gene)
        return Individual(result_position)

#### Implementacja etapów krzyżowania i mutacji

In [6]:
def crossover_phase(population, probability):
    temp_population = []
    for individual in population:
        if random.random()<probability:
            other = random.choice(population)
            temp_population.append(individual.crossover(other))
        else:
            temp_population.append(individual)
    return temp_population


def mutate_phase(population,mutation_strentgth):
    temp_population = []
    for individual in population:
        individual.mutate(mutation_strentgth)
        temp_population.append(individual)
    return temp_population

### Generowanie populacji początkowej
Z racji, że znamy w przybliżeniu funkcje możemy ustawić pewne ograniczenia jeśli chodzi o generowane punkty początkowe, interesuje nas przedział $x,y \in [-5,5] \times [-5,5]$, ważne jest natomiast wybranie rozmiaru populajci, za mała może utrudnić przeszukiwanie przestrzenie, ale za duża znacznie spowolni cały proces. 

In [7]:
def generate_population(size:int,restrictions):
    population = list()
    for _ in range(size):
        # Losowanie punktu początkowego przestrzegając ograniczeń
        position = [random.uniform(restrictions[i][0],restrictions[i][1]) for i in range(len(restrictions))]
        population.append(Individual(position))
    return population


### Ocena populacji 
Przed podejmowanie decyzji o dalszym wyborze osobników, potrzeba możliwości uczciwego porównywania ich, do tego przyda się funckja oceny populacji. W tym przypadku minimalizujemy funkcje, więc sortujemy osobniki rosnąco w populacji. 

In [8]:
def rate_individuals(population,func):
    for individual in population:
        individual.evaluate(func)
    population.sort(key=lambda ind:ind.value)

In [9]:
def evolutionary_algorithm(func,start_population,iteration_time,mutation_strength,crossover_probability):
    population = deepcopy(start_population)
    rate_individuals(population,func)
    best_individual = deepcopy(population[0])
    trace = {}
    for step in range(iteration_time):
        trace[step] = population
        population = tournament_selection(population,2)
        population = crossover_phase(population,crossover_probability)
        population = mutate_phase(population,mutation_strength)
        rate_individuals(population,func)
        succession_phase(population,best_individual)
        if population[0]<best_individual:
            best_individual = deepcopy(population[0])
    return trace